In [11]:
import random
import datetime
from dateutil.relativedelta import relativedelta
from faker import Faker
random.seed(49)
fake = faker.Faker(seed=49,locale="es")

1- Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota,fecha). Se pide resolver utilizando PySpark:

A. Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.

B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón,promedio).

C. El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido).

In [35]:
cantidad_alumnos = 20
padrones = [ x for x in range(1, cantidad_alumnos+1) ]
materias = ("ALGEBRA I", "ANALISIS I", "ALGEBRA II", "ANALISIS II", "PROBABILIDAD", "ALGORITMOS I", "ALGORITMOS II")
notas = list(range(1,11))
fechas = ("01-02-2019", "01-07-2019", "01-02-2018", "01-07-2018", "01-12-2018", "01-02-2017", "01-07-2017", "01-12-2017")
def alumno_random(): return tuple([random.choice(x) for x in (padrones, materias, notas, fechas)])

cantidad_registros = 50
notas = { alumno_random() for a in range(cantidad_registros) }

rdd_notas = sc.parallelize(notas)

def es_fecha_ultimos_dos_anios(fecha):
    hoy = datetime.datetime.today()
    d,m,y = fecha.split("-")
    fecha = datetime.date(int(y), int(m), int(d))
    anios = relativedelta(hoy,fecha).years
    return anios <= 2

es_aprobado = lambda x: x > 3

ejercicio_a = rdd_notas\
    .filter(lambda x: es_aprobado(x[2]))\
    .filter(lambda x: es_fecha_ultimos_dos_anios(x[3]))\
    .map(lambda x: (x[0], 0))\
    .groupByKey()\
    .count()

print("{} alumnos de {} aprobaron al menos 1 materia en los últimos 2 años".format(ejercicio_a, cantidad_alumnos))

ejercicio_b = rdd_notas\
    .map(lambda x: (x[0], (x[2], 1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+1))\
    .map(lambda x: (x[0], x[1][0]/x[1][1]))
# pide el RDD, no ejecuto ninguna acción
print(ejercicio_b)

nombres = [(x, fake.first_name(),fake.last_name()) for x in padrones]
rdd_nombres = sc.parallelize(nombres)

padron_mejor_promedio = ejercicio_b.max(lambda x: x[1])[0]
ejercicio_c = rdd_nombres.filter(lambda x: x[0]==padron_mejor_promedio).collect()[0]
print("{}, {} (padron {}) es el alumno con mejor promedio".format(ejercicio_c[2], ejercicio_c[1], ejercicio_c[0]))

15 alumnos de 20 aprobaron al menos 1 materia en los últimos 2 años
PythonRDD[76] at RDD at PythonRDD.scala:53
Marquez, Carolina (padron 17) es el alumno con mejor promedio
